In [1]:
#Monter Google Drive et installer les bibliothèques
from google.colab import drive
drive.mount('/content/drive')

!pip install rasterio fiona shapely geopandas matplotlib


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 51.8 MB/s eta 0:00:00


In [2]:
#Configuration des chemins de fichiers

import os
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
import geopandas as gpd
import matplotlib.pyplot as plt

base_folder = '/content/drive/MyDrive/Stage_CEFREM_Salma_2025/Data'
mnt_raw_tiles_folder = os.path.join(base_folder, 'MNT', 'dossierOriginalTuiles')
pyrenees_polygon_path = os.path.join(base_folder, 'GIS', 'polygon_PYRENEES.shp')
output_merged_mnt = os.path.join(base_folder, 'MNT', "Pyrenees_SRTM_30m_Merged.tif")
output_clipped_mnt = os.path.join(base_folder, 'MNT', "Pyrenees_MNT_30m_Final_Clipped.tif")


In [ ]:
#Fusionner les tuiles SRTM

input_files = [os.path.join(mnt_raw_tiles_folder, f) for f in os.listdir(mnt_raw_tiles_folder) if f.endswith(('.tif', '.hgt'))]

if not input_files:
    print("Aucune tuile détectée.")
else:
    src_files_to_mosaic = []
    try:
        for fp in input_files:
            src_files_to_mosaic.append(rasterio.open(fp))

        mosaic, out_transform = merge(src_files_to_mosaic)
        out_meta = src_files_to_mosaic[0].meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_transform,
            "crs": src_files_to_mosaic[0].crs,
            "nodata": src_files_to_mosaic[0].nodata
        })

        with rasterio.open(output_merged_mnt, "w", **out_meta) as dest:
            dest.write(mosaic)

        print(f"MNT fusionné : {output_merged_mnt}")
    except Exception as e:
        print(f"Erreur de fusion : {e}")
    finally:
        for src in src_files_to_mosaic:
            src.close()


In [ ]:
#Affichage du MNT fusionné

if os.path.exists(output_merged_mnt):
    with rasterio.open(output_merged_mnt) as src:
        downsample = 4
        mnt_data = src.read(1, out_shape=(1, src.height // downsample, src.width // downsample))
        extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

        fig, ax = plt.subplots(figsize=(10, 10))
        im = ax.imshow(mnt_data, extent=extent, cmap='terrain', origin='upper')
        plt.colorbar(im, label='Altitude (m)')

        gdf = gpd.read_file(pyrenees_polygon_path)
        if gdf.crs != src.crs:
            gdf = gdf.to_crs(src.crs)
        gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)

        ax.set_title("MNT Fusionné avec Emprise des Pyrénées")
        plt.show()
else:
    print("Fichier MNT fusionné introuvable.")


In [ ]:
#Découpage du MNT
try:
    gdf = gpd.read_file(pyrenees_polygon_path)
    shapes = gdf.geometry.tolist()

    with rasterio.open(output_merged_mnt) as src:
        if gdf.crs != src.crs:
            gdf = gdf.to_crs(src.crs)
            shapes = gdf.geometry.tolist()

        out_image, out_transform = mask(src, shapes, crop=True, nodata=src.nodata)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "crs": src.crs,
            "nodata": src.nodata
        })

        with rasterio.open(output_clipped_mnt, "w", **out_meta) as dest:
            dest.write(out_image)

    print(f"MNT découpé : {output_clipped_mnt}")
except Exception as e:
    print(f"Erreur de découpage : {e}")


In [ ]:
#Affichage du MNT découpé final

try:
    with rasterio.open(output_clipped_mnt) as src:
        mnt_data = src.read(1)
        extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

        fig, ax = plt.subplots(figsize=(10, 10))
        im = ax.imshow(mnt_data, extent=extent, cmap='terrain', origin='upper')
        plt.colorbar(im, label='Altitude (m)')

        gdf = gpd.read_file(pyrenees_polygon_path)
        if gdf.crs != src.crs:
            gdf = gdf.to_crs(src.crs)
        gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)

        ax.set_title("MNT Final des Pyrénées (Découpé)")
        plt.show()
except Exception as e:
    print(f"Erreur affichage final : {e}")
